# Theory

This file contains a summary of theory used in this project. It does not aim to provide a comprehensive introduction to the Finite Element Method (FEM) but may provide some useful clarification.

## The Problem

This code is designed to solve the steady-state diffusion equation in a 2-dimensional domain using the FEM. The diffusion equation is relevant in a number of scenarios including thermal diffusion, the diffusion of a chemical in a medium or the spatial distributions of neutrons in a nuclear reactor.

We will assume the diffusion coefficient $D$ is a constant and so we may write the equation we want to solve as:

\begin{align}
-D\nabla^{2}\phi(x, y) = S ,
\end{align}

where $\phi(x, y)$ is the field being solved for (e.g. the temperature of an object, concentration of a chemical of density of neutrons) and $S$ is a source which we will assume is constant over the domain

We will further define a zero boundary-condition such that $\phi = 0$ at the boundary of the domain.

We want to write a code which will be able to read an input defining a mesh made of triangular and/or quadrilateral elements, receive some values for the diffusion coefficient and the source and then solve the system and output the solution to the equation.

## The Finite Element Method

The finite element method breaks the domain up into a number of elements. Each type of element (for example triangle and quadrilateral elements) has several properties defined over a canonical element defined in local coordinates. Each element in the domain will then have coordiantes provided in the physical space of that domain. For example, take the triangular element below:

<center><img src='https://raw.githubusercontent.com/coolernato/2D-FEM-Python/master/Github_resources/traingle_geometry.png' />
<figcaption>Canonical traingle element and world-sapce triangle element</figcaption></center>
</figure>

The image on the left shows the canconical triangular element. In this image P$_{1}$ is at (0,0) in the ($\xi$, $\eta$) coordinate system, P$_{2}$ is at (1,0) and P$_{3}$ is at (0,1).The diagram on the right shows a triangluar element in world space. The angles at each vertex and the rotation of the traingle may be different to the canonical triangle.

Each canonical element defines a number of shape functions over the extent of the element in local coordinates. For example, for a traingular element, they are defined as follows:

$$
\begin{align}
N_{1}(\xi, \eta) &= 1 - \xi - \eta,\\
N_{2}(\xi, \eta) &= \xi,\\
N_{3}(\xi, \eta) &= \eta.
\end{align}
$$

Note that $N_{i} = 1$ at P$_{i}$ but is zero at P$_{j}$ for $j\ne i$. In addition $\sum\limits_{i} N_{i}(\xi, \eta) = 1$ for all $\xi$ and $\eta$. These shape functions are zero outside of the element.

We then approximate the solution in the domain as:

\begin{align}
\phi(x,y) = \sum\limits_{i} \phi_{i} \psi_{i}(x,y)
\end{align}

where $\phi_{i}$ is the weight of the shape function and $\psi_{i}(x,y)$ is the $i$th shape function. We will be using the first-order continuous FEM which means the weights are the magnitudes of the solution at the nodes of the system. Further, the $i$th shape function is the sum of the shape functions of each element which touch node $i$ that are 1 are the corner of the element which touches node $i$

The FEM then proceeds to try to calcualte the values of $\phi_{i}$ to complete the appximation of the solution.

We proceed by approximating the original equation of the problem in terms of the basis functions and weights at each node. From this a number of simultaneous equations for the weights can be formed. This can be represented as a matrix equation which can be solved by a variety of methods.

## Forming the Equations

We begin by taking the original equation

\begin{align}
-D\nabla^{2}\phi(x, y) = S
\end{align}

and then multiply both sides of the equation by a test function and integrate this equation over the domain. We will be used the Galerkin approximation, meaning we also use the basis functions of the element as test functions. Thus, using the $j$th basis function as a test function, in each element we obtain the equation:

\begin{align}
-\int_{\Omega}\psi_{j}(x,y)\nabla^{2}\phi(x,y)\textrm{d}A = \int_{\Omega}\psi_{j}(x,y)S\textrm{d}A,
\end{align}

where the subscript $\Omega$ of the integral and the $\textrm{d}A$ indicates the integrals are over the interior of the element (in our 2D example, this is the area of each element). We then use Green's First Identity to obtain:

\begin{align}
\int_{\Omega}\nabla\psi_{j}(x,y)\cdot\nabla \phi(x,y) \textrm{d}A - \int_{\partial\Omega} \psi_{j}(x,y)\left(\nabla\phi(x,y)\right)\cdot \vec{n} \textrm{d}B = \int_{\Omega}\psi_{j}(x,y)S \textrm{d}A,
\end{align}

where the subscript $\partial\Omega$ of the integral and the $\textrm{d}B$ indicates the final integral on the left hand side of the equation is over the boundary of the domain (for our 2D example this is a line integral around the boundary of domain). $\vec{n}$ is the normal to the boundary of the element.

Next, we substitute in the approximation of the solution

\begin{align}
\phi(x,y) = \sum\limits_{i} \phi_{i} \psi_{i}(x,y)
\end{align}

to obtain:

\begin{align}
\sum\limits_{i} \phi_{i} \left(\int_{\Omega}\nabla\psi_{j}(x,y)\cdot\nabla \psi_{i}(x,y) \textrm{d}A - \int_{\partial\Omega} \psi_{j}(x,y)\left(\nabla\psi_{i}(x,y)\right)\cdot \vec{n} \textrm{d}B \right) = \int_{\Omega}\psi_{j}(x,y)S \textrm{d}A,
\end{align}

The next step is to form a matric equation of the form $\mathbf{M}\vec{\phi} = \vec{S}$ where each row of the matrix corresponds to this equation for one value of $j$. The vector $\vec{\phi}$ contains the values of $\phi_{i}$ - the weights of each of the basis functions. Thus calculating $\vec{\phi}$ is sufficient to calculate the approximate solution to the original equation. We form this matrix equation by calcualting the contribution to each term for each equation for each element in the domain and adding them together.

### The Jacobian

The Jacobian $\boldsymbol{J}$ of an element is useful when transforming between a real element and a canonical element. It may be defined as:

$
\boldsymbol{J} = \left(\begin{matrix} \frac{\partial x}{\partial \xi} & \frac{\partial x}{\partial \eta}\\
\frac{\partial y}{\partial \xi} & \frac{\partial y}{\partial \eta}\end{matrix}\right).
$

To calculate this, we start with the definition of the real world location $\hat{x} = (x, y)$ of a location within an element given its local coordinate location $\hat{\xi} = (\xi, \eta)$:

$\hat{x} = \sum\limits_{i}N_{i}(\hat{\xi})\hat{x}_i$

where $N_{i}(\hat{\xi})$ is the $i$th shape function of the element evaluated at the relevant local coordinate and $\hat{x}_i = (x_{i}, y_{i})$ is the global coordinate of the $i$th node of the element. To get the elements of the Jacobian, we may differentiate this expression by $\xi$ or $\eta$ to get:

$
\boldsymbol{J} = \left(\begin{matrix} \sum\limits_{i}\frac{\partial N_{i}}{\partial \xi}(\hat{\xi})x_{i} & \sum\limits_{i}\frac{\partial N_{i}}{\partial \eta}(\hat{\xi})x_{i} \\
\sum\limits_{i}\frac{\partial N_{i}}{\partial \xi}(\hat{\xi})y_{i} & \sum\limits_{i}\frac{\partial N_{i}}{\partial \eta}(\hat{\xi})y_{i}\end{matrix}\right).
$

For many element types this means the Jacobian will be a function of the location within the element. For some elements, like the first-order triangular element we discussed earlier, the result will be a constant matrix that doesn't vary with location within the element. For a first-order triangle, this expression simplifies to:

$
\boldsymbol{J}_{tri} = \left(\begin{matrix} x_{2} - x_{1} & x_{3} - x_{1} \\
y_{2} - y_{1} & y_{3} - y_{1} \end{matrix}\right).
$


## Integrating a Function Over and Element

It's possible to approximate an integrate over a function $f$ over an element using a process known as Gaussian quadrature as follows:

$ \int_{E} f(\hat{x}) \textrm{d}A = \sum\limits_{j} f(\hat{\xi}_{j})W_{j}|\boldsymbol{J}|(\hat{\xi})$

where $|\boldsymbol{J}|(\hat{\xi})$ is the determinant of the Jacobian and $\hat{\xi}_{j}$ and $W_{j}$ are the local coordinates and weights of the Gaussian quadrature scheme for the element. These points and weights are defined for each element such that the above approximation is as accurate as possible for a given number of points. This scheme has different orders, with increasing numbers of points. For an example, the first few orders for Gaussian quadrature on a triangular element are given below:

<table>
    <thead>
        <tr>
            <th>Order</th>
            <th>$\xi_{j}$</th>
            <th>$\eta_{j}$</th>
            <th>$W_{j}$</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>0</td>
            <td>$\frac{1}{3}$</td>
            <td>$\frac{1}{3}$ </td>
            <td>$0.5$ </td>
        </tr>
        <tr>
            <td rowspan=3>1</td>
            <td>$0.5$</td>
            <td>$0$ </td>
            <td>$\frac{1}{6}$ </td>
        </tr>
        <tr>
            <td>$0$ </td>
            <td>$0.5$ </td>
            <td>$\frac{1}{6}$ </td>
        </tr>
        <tr>
            <td>$0.5$</td>
            <td>$0.5$ </td>
            <td>$\frac{1}{6}$ </td>
        </tr>
        <tr>
            <td rowspan=4>2</td>
            <td>$\frac{1}{3}$</td>
            <td>$\frac{1}{3}$ </td>
            <td>$-\frac{27}{96}$ </td>
        </tr>
        <tr>
            <td>$0.6$</td>
            <td>$0.2$ </td>
            <td>$\frac{25}{96}$ </td>
        </tr>
        <tr>
            <td>$0.6$</td>
            <td>$0.2$ </td>
            <td>$\frac{25}{96}$ </td>
        </tr>
        <tr>
            <td>$0.2$</td>
            <td>$0.2$ </td>
            <td>$\frac{25}{96}$ </td>
        </tr>
    </tbody>
</table>

### Term 1

The first term is:

\begin{align}
\sum\limits_{i} \phi_{i} \int_{\Omega}\nabla\psi_{j}(x,y)\cdot\nabla \psi_{i}(x,y) \textrm{d}A.
\end{align}

Writing out the derivatives in full:

\begin{align}
\sum\limits_{i} \phi_{i} \int_{\Omega} \left(\frac{\partial \psi_{j}(x,y)}{\partial x}\frac{\partial \psi_{i}(x,y)}{\partial x} + \frac{\partial \psi_{j}(x,y)}{\partial y}\frac{\partial \psi_{i}(x,y)}{\partial y} \right)\textrm{d}A.
\end{align}

We may re=write the derivative of a shape function as follows:

$
\frac{\partial \psi_{i}}{\partial x} = \frac{\partial \psi_{i}}{\partial \xi}\frac{\partial \xi}{\partial x}

## Further Reading

This is a very brief description of the method and aims to serve mainly as a disambiguation for readers who are already familiar with the FEM. For a fuller description of the FEM, consider the following resources:

- COMSOL have written a [more detailed and rigorous description](https://uk.comsol.com/multiphysics/finite-element-method#weak2) of the FEM being used to solve the diffusion equation.
- Some [good notes](http://users.metu.edu.tr/csert/me582/ME582%20Ch%2003.pdf) on the 2D FEM method